## Imports


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from functools import partial

from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.aruco.sheet_aruco_config import SheetArucoConfig
from snap_fit.grid.grid_model import GridModel
from snap_fit.grid.scoring import score_grid_with_details
from snap_fit.grid.types import GridPos
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece_matcher import PieceMatcher
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager

# Import from the new solver module
from snap_fit.solver import NaiveLinearSolver
from snap_fit.solver import partition_pieces_by_type
from snap_fit.solver.utils import infer_grid_size

## Load Puzzle Data


In [ ]:
# Configure ArUco detection
board_config = ArucoBoardConfig(
    markers_x=7,
    markers_y=5,
    marker_length=100,
    marker_separation=100,
)
detector_config = ArucoDetectorConfig(board=board_config)

# Initialize sheet loader
sheet_config = SheetArucoConfig(detector=detector_config)
sheet_aruco = SheetAruco(sheet_config)
aruco_loader = sheet_aruco.load_sheet

# Load puzzle sheets
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "sample_puzzle_v1" / "sheets"
lg.info(f"Loading data from {data_dir}")

manager = SheetManager()
manager.add_sheets(
    folder_path=data_dir,
    pattern="*.png",
    loader_func=aruco_loader,
)

lg.info(
    f"Loaded {len(manager.get_sheets_ls())} sheets with "
    f"{len(manager.get_pieces_ls())} pieces"
)

## Partition Pieces and Infer Grid Size


In [ ]:
# Partition pieces by type using the utility function
corners, edges, inners = partition_pieces_by_type(manager)
rprint(f"Partitioned: {len(corners)} corners, {len(edges)} edges, {len(inners)} inners")

# Infer grid size from piece counts
grid_size = infer_grid_size(corners, edges, inners, tolerance=4)
if grid_size:
    rows, cols = grid_size
    rprint(f"Inferred grid size: {rows}x{cols}")
else:
    rows, cols = 6, 8  # Default fallback
    rprint(f"Could not infer grid size, using default: {rows}x{cols}")

## Initialize Matcher


In [ ]:
# Initialize matcher and pre-compute all pairwise scores
matcher = PieceMatcher(manager)
matcher.match_all()
lg.info(f"Cached {len(matcher.results)} segment pair scores")

## Run the Solver


In [ ]:
# Create grid model
grid = GridModel(rows=rows, cols=cols)
rprint(grid)

# Initialize and run solver
solver = NaiveLinearSolver(
    grid=grid,
    matcher=matcher,
    manager=manager,
    corners=corners,
    edges=edges,
    inners=inners,
)

result_state = solver.solve()

## Evaluate Results


In [ ]:
# Score the solution
total_score, edge_scores = score_grid_with_details(result_state, matcher)

rprint("[bold green]Solver Results[/bold green]")
rprint(f"  Pieces placed: {result_state.placed_count}/{grid.total_cells}")
rprint(f"  Total score: {total_score:.4f} (lower is better)")
rprint(f"  Edges scored: {len(edge_scores)}/{grid.total_edges}")

# Edge score statistics
scores = list(edge_scores.values())
if scores:
    rprint("\n[bold blue]Edge Score Statistics[/bold blue]")
    rprint(f"  Min score: {min(scores):.4f}")
    rprint(f"  Max score: {max(scores):.4f}")
    rprint(f"  Mean score: {sum(scores) / len(scores):.4f}")

## Display Placement Grid


In [ ]:
from pathlib import Path

rprint("[bold blue]Placement Grid[/bold blue]")
for ro in range(grid.rows):
    row_str = ""
    for co in range(grid.cols):
        pos = GridPos(ro=ro, co=co)
        placement = result_state.get_placement(pos)
        if placement:
            piece_id, orientation = placement
            orient_str = f"{orientation.value:>3}°"
            sheet_id_num = int(Path(piece_id.sheet_id).stem.split("_")[-1])
            row_str += f"[{sheet_id_num:>2}:{piece_id.piece_id:>2}@{orient_str}] "
        else:
            row_str += "[  empty  ] "
    rprint(row_str)

## Visualize Solved Puzzle


In [ ]:
import cv2
import numpy as np

from snap_fit.grid.orientation import Orientation
from snap_fit.grid.placement_state import PlacementState
from snap_fit.image.utils import show_image_mpl


def render_solved_puzzle(
    state: PlacementState,
    manager: SheetManager,
    cell_size: int = 150,
) -> np.ndarray:
    """Render the solved puzzle as a composite image."""
    grid = state.grid
    img_h = grid.rows * cell_size
    img_w = grid.cols * cell_size
    canvas = np.zeros((img_h, img_w, 3), dtype=np.uint8)

    for ro in range(grid.rows):
        for co in range(grid.cols):
            pos = GridPos(ro=ro, co=co)
            placement = state.get_placement(pos)
            if placement is None:
                continue

            piece_id, orientation = placement
            piece = manager.get_piece(piece_id)
            if piece is None:
                continue

            # Get piece image and rotate
            piece_img = piece.img_orig.copy()
            if orientation == Orientation.DEG_90:
                piece_img = cv2.rotate(piece_img, cv2.ROTATE_90_CLOCKWISE)
            elif orientation == Orientation.DEG_180:
                piece_img = cv2.rotate(piece_img, cv2.ROTATE_180)
            elif orientation == Orientation.DEG_270:
                piece_img = cv2.rotate(piece_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            piece_resized = cv2.resize(piece_img, (cell_size, cell_size))

            y_start = ro * cell_size
            x_start = co * cell_size
            canvas[y_start : y_start + cell_size, x_start : x_start + cell_size] = (
                piece_resized
            )

    return canvas


puzzle_img = render_solved_puzzle(result_state, manager, cell_size=200)
show_image_mpl(puzzle_img, figsize=(12, 9))